In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


# 인식할 손동작 클래스들을 정의
actions = ['one', 'two', 'three']
seq_length = 30

# 미리 훈련된 모델을 불러옴
model = load_model('models/model.keras')

# MediaPipe hands 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 웹캠을 사용하여 비디오 캡처를 시작함
cap = cv2.VideoCapture(0)

# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

seq = [] # 손동작 시퀀스를 저장할 리스트
action_seq = [] # 인식된 손동작 시퀀스를 저장할 리스트

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1) # 이미지를 좌우 반전
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # OpenCV의 BGR 이미지를 RGB 이미지로 변환
    result = hands.process(img) # MediaPipe Hands 모델을 사용하여 손을 감지
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # RGB 이미지를 다시 BGR 이미지로 변환
    
    # 손을 감지한 경우
    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # 관절 간의 각도를 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            d = np.concatenate([joint.flatten(), angle])

            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            # 손동작 시퀀스의 길이가 지정된 길이보다 작은 경우
            if len(seq) < seq_length:
                continue

            # 입력 데이터를 준비
            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

            # 모델을 사용하여 손동작을 예측
            y_pred = model.predict(input_data).squeeze()

            # 가장 높은 확률을 가진 클래스를 찾음
            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]

            # 예측 확률이 일정 임계값 이상인 경우에만 인식
            if conf < 0.9:
                continue

            # 예측된 손동작을 저장
            action = actions[i_pred]
            action_seq.append(action)

            # 인식된 손동작 시퀀스의 길이가 3 미만인 경우에는 무시
            if len(action_seq) < 3:
                continue

            this_action = '?' # 기본값으로 물음표를 설정
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action = action

            # 해당 동작을 이미지에 텍스트로 표시
            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    # out.write(img0)
    # out2.write(img)
    # 이미지를 화면에 표시
    cv2.imshow('img', img)
    # 키 입력을 대기하며 'q' 키를 누르면 프로그램을 종료
    if cv2.waitKey(1) == ord('q'):
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [2]:
# 웹캠 정상 작동 테스트
# 실행해서 웹캠 잘 나오면 cv 웹캠은 이상 없는 것

import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('Camera Test', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
